In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from getpass import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
# RetrievalQA : LangChain에서 지원하는 질의 응답 수행 클래스
from langchain.chains import RetrievalQA
# ChatOpenAI : LangChain에서 지원하는 OpenAI 대화 전용 모델
# 기존의 completion 객체는 입력 형태가 맞지 않아 RetrievalQA에 직접 활용할 수 없음
from langchain.chat_models import ChatOpenAI
# CohereRerank : LangChain에서 지원하는 Reranking 구현 클래스
from langchain.retrievers.document_compressors import CohereRerank
# ContextualCompressionRetriever : 검색 된 문서의 문맥을 압축하여 보다 집중적이고 관련성 높은 정보를 제공하는 검색기 (Rerank 객체와 결합 가능)
# 문맥 압축은 가장 관련성이 높은 부분을 선택하고 덜 중요한 정보는 삭제하여 핵심만 남겨두는 것
from langchain.retrievers import ContextualCompressionRetriever
# LangChain과 연동하여 Chroma DB의 기능을 약식으로 구현할 수 있게 해주는 Chroma 클래스
from langchain.vectorstores import Chroma
# OpenAI 지원 사전 학습된 임베딩 모델 임포트
from langchain.embeddings.openai import OpenAIEmbeddings

In [2]:
# OpenAI 객체 생성
MY_API_KEY = getpass("OpenAI API Key >> ")

In [3]:
loaders = [CSVLoader(file_path = "../data/credit_result.csv", encoding = "utf-8"), CSVLoader(file_path = "../data/chk_result.csv", encoding = "utf-8")]

csv_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

my_chunks = []
for i in loaders :
    # 2개의 파일들로부터 추출 된 각각의 document 객체들을 하나의 리스트에 병합
    my_chunks.extend(i.load_and_split(text_splitter = csv_splitter))
    
print(len(my_chunks))
my_chunks

1365


[Document(page_content="name: 신한카드 Mr.Life\ncard_company: 신한카드\nbenefit: ['월납요금(공과금) 10% 할인서비스', '편의점 10% 할인', '병원/약국업종 10% 할인', '세탁소 업종 10% 할인', '온라인 쇼핑 10% 할인 ', '택시 10% 할인 ', '식음료 10% 할인', '3대 마트 10% 할인', '4대 정유사 리터당 60원 할인', ' 인테이크몰 20% 할인']", metadata={'source': '../data/credit_result.csv', 'row': 0}),
 Document(page_content="name: 삼성카드 & MILEAGE PLATINUM (스카이패스)\ncard_company: 삼성카드\nbenefit: ['모든 가맹점 이용금액 1,000원당 1마일리지 기본적립', '백화점 이용금액 1,000원당 2마일리지 특별적립', '주유 이용금액 1,000원당 2마일리지 특별적립', '커피 이용금액 1,000원당 2마일리지 특별적립', '편의점 이용금액 1,000원당 2마일리지 특별적립', '택시 이용금액 1,000원당 2마일리지 특별적립', '인천공항 라운지 본인 무료 이용', '인천공항 발렛파킹 무료 이용', '공항 커피 서비스', '아멕스 PLATINUM 등급 서비스', '여행의 설레임을 담은 디자인으로 리뉴얼 된 삼성카드 & MILEAGE PLATINUM']", metadata={'source': '../data/credit_result.csv', 'row': 1}),
 Document(page_content="name: KB국민 My WE:SH 카드\ncard_company: KB국민카드\nbenefit: ['[나한테 진심 서비스] KB Pay 10% 할인', '[나한테 진심 서비스] 음식점, 편의점 10% 할인', '[나한테 진심 서비스] 이동통신요금 10% 할인', '[나한테 진심 서비스] OTT 30% 할인', '[더욱 진심 서비스] 먹는데 진심-배달/

In [4]:
# OpenAI에서 지원해 주는 사전 학습된 임베딩 모델을 적용한 객체 생성
my_embedding = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    api_key = MY_API_KEY
)

# DB 경로 설정
my_directory = "../data/VectorStores/"

# from_documents() : 청킹된 문서(documents 객체)의 정보 및 임베딩, 경로 정보로 Chroma DB 생성
vectordb = Chroma.from_documents(
    documents = my_chunks,
    embedding = my_embedding,
    persist_directory = my_directory
)

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
# as_retriever() : 정의된 벡터 DB를 정보 검색기로 사용할 수 있게 해주는 함수 (k = 출력 청크 갯수)
my_retriever = vectordb.as_retriever(search_kwargs = {"k" : 5})

chat_model = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    openai_api_key = MY_API_KEY
)

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
# Reranking 수행
COHERE_API_KEY = getpass("Cohere API KEY >> ")

# CohereRerank 객체 생성
my_rerank = CohereRerank(
    cohere_api_key = COHERE_API_KEY,
)

# 검색 시 객체 설정
compression_retriever = ContextualCompressionRetriever(
    base_compressor = my_rerank,
    base_retriever = my_retriever
)

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(


In [8]:
qna = RetrievalQA.from_chain_type(
    llm = chat_model,
    # chain_type = 답변을 위해 청크들이 결합되는 타입을 지정
    # 1. stuff : 검색된 청크들의 내용을 간단히 결합하여 응답 생성
    # 2. map_reduce : 청크들을 각각 처리한 후 개별 결과를 결합하여 응답 생성 (큰 규모 데이터)
    # 3. refine : 이전 청크의 처리 결과를 다음 청크에 반영하고 지속적으로 개선하여 응답 생성
    chain_type = "refine",
    retriever = compression_retriever # rerank 기능 포함
)

prompt_1shot_persona1 = """
당신은 카드 설계사입니다. 사용자 정보들이 주어졌을 때 최적의 카드를 추천하는 것이 당신의 목표입니다.
DB에 정의된 필드는 다음과 같습니다.
name: 카드 명
card_company: 카드사 명
benefit: 혜택

다음은 당신이 참고할 예시 정보와 그에 대한 카드 추천 예시 입니다.

예시 정보:
대중교통(지하철, 버스, 택시) 출퇴근
아침식사는 편의점, 점심은 음식점, 저녁은 외식(레스토랑)
운동을 즐겨함, 헬스장을 다님
월 평균 통신비: 8만원
주말에는 주로 친구들과 맛집 탐방을 즐김
월 평균 쇼핑비: 20만원

예시 결과:
name: **XX 카드**,
card_company: **XX 은행**,
benefit: **대중교통(버스, 지하철) 이용 시 10% 청구할인, 통신사 요금 결제 시 SKT, KT Olleh, LG U+ 10% 청구할인** 등의 혜택이 있습니다.

아래 정보에 대해 카드를 한국어로 추천해주세요:
생활 패턴:
평일에 대중교통(지하철)으로 출퇴근
출퇴근 시간에 맞춰 아침 일찍 일어나고, 저녁에는 헬스장을 다님
소비 패턴:
월 평균 통신비: 8만원
주말에는 주로 친구들과 맛집 탐방을 즐김
간혹 해외여행을 즐기며, 여행 경비는 주로 카드로 결제
월 평균 쇼핑비: 20만원
니즈:
교통비 할인 혜택
맛집 및 외식 할인 혜택
통신비 할인 혜택
"""

print(qna.run(prompt_1shot_persona1))

C:\Dev\tools\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Based on the additional context provided about the DA@카드의정석 from 우리카드, it aligns well with the user's needs for transportation, dining, and communication expense discounts. The card offers a wide range of benefits including discounts at various merchants, free airport lounge access, and discounts on transportation, dining, and communication expenses. Therefore, the original recommendation of DA@카드의정석 from 우리카드 remains suitable and does not require further refinement.


In [9]:
prompt_1shot_persona2 = """
당신은 카드 설계사입니다. 사용자 정보들이 주어졌을 때 최적의 카드를 추천하는 것이 당신의 목표입니다.
DB에 정의된 필드는 다음과 같습니다.
name: 카드 명
card_company: 카드사 명
benefit: 혜택

다음은 당신이 참고할 예시 정보와 그에 대한 카드 추천 예시 입니다.

예시 정보:
대중교통(지하철, 버스, 택시) 출퇴근
아침식사는 편의점, 점심은 음식점, 저녁은 외식(레스토랑)
운동을 즐겨함, 헬스장을 다님
월 평균 통신비: 8만원
주말에는 주로 친구들과 맛집 탐방을 즐김
월 평균 쇼핑비: 20만원

예시 결과:
name: **XX 카드**,
card_company: **XX 은행**,
benefit: **대중교통(버스, 지하철) 이용 시 10% 청구할인, 통신사 요금 결제 시 SKT, KT Olleh, LG U+ 10% 청구할인** 등의 혜택이 있습니다.

아래 정보에 대해 카드를 한국어로 추천해주세요:
생활 패턴:
평일에는 주로 아이들 학교 보내고, 집안일을 함
주말에는 가족들과 함께 시간을 보냄
소비 패턴:
월 평균 식료품비: 50만원
아이들 학원비 및 교육비: 30만원
패밀리 레스토랑 및 가족 외식 빈도 높음
생활용품 및 가전제품 쇼핑 빈도 높음
니즈:
식료품 할인 혜택
학원비 및 교육비 할인 혜택
패밀리 레스토랑 할인 혜택
생활용품 및 가전제품 구매 할인 혜택
"""

print(qna.run(prompt_1shot_persona2))

**다담카드**

카드사: KB국민카드

혜택:
- 버스, 지하철 이용 시 10% 청구할인
- SKT, KT Olleh, LG U+ 요금 결제 시 10% 청구할인
- SK주유소(충전소)에서 리터당 60원 청구할인
- 인터파크 영화 예매 시 1매당 3,500원 청구할인
- 롯데월드, 에버랜드, 캐리비안베이 입장료 30~50% 할인
- 해외가맹점에서 5% 할인캐시백
- 선택 적립 서비스팩 중 택 1: 생활/교육/쇼핑/직장인/레저팩
- 연간 이용에 따른 다담리워드 서비스 및 함께하는 기부 서비스

**다담카드**는 다양한 할인 혜택과 적립 서비스를 제공하여 가정에서의 지출을 절약하고 다양한 혜택을 누릴 수 있도록 도와줍니다.생활 패턴과 소비 패턴을 고려하여 다양한 혜택을 제공하여 가정에서의 지출을 절약하고 가족과 함께 보다 풍성한 시간을 보낼 수 있도록 도와줍니다.


In [10]:
prompt_1shot_persona3 = """
당신은 카드 설계사입니다. 사용자 정보들이 주어졌을 때 최적의 카드를 추천하는 것이 당신의 목표입니다.
DB에 정의된 필드는 다음과 같습니다.
name: 카드 명
card_company: 카드사 명
benefit: 혜택

다음은 당신이 참고할 예시 정보와 그에 대한 카드 추천 예시 입니다.

예시 정보:
대중교통(지하철, 버스, 택시) 출퇴근
아침식사는 편의점, 점심은 음식점, 저녁은 외식(레스토랑)
운동을 즐겨함, 헬스장을 다님
월 평균 통신비: 8만원
주말에는 주로 친구들과 맛집 탐방을 즐김
월 평균 쇼핑비: 20만원
예시 결과:
name: **XX 카드**,
card_company: **XX 은행**,
benefit: **대중교통(버스, 지하철) 이용 시 10% 청구할인, 통신사 요금 결제 시 SKT, KT Olleh, LG U+ 10% 청구할인** 등의 혜택이 있습니다.

아래 정보에 대해 카드를 한국어로 추천해주세요:
생활 패턴:
평일에는 주로 학교 수업과 과제, 동아리 활동
주말에는 친구들과 여가 시간을 보냄
아르바이트를 하며 생활비를 충당
소비 패턴:
월 평균 교통비: 5만원
편의점 및 카페 이용 빈도 높음
온라인 쇼핑을 자주 함
취미 생활(영화, 공연 관람 등)에 지출
니즈:
교통비 할인 혜택
편의점 및 카페 할인 혜택
온라인 쇼핑 할인 혜택
취미 생활 관련 할인 혜택
"""

print(qna.run(prompt_1shot_persona3))

추천 카드:
- 카드 명: **KB국민 우리동네 체크카드**
- 카드사 명: **KB국민카드**
- 혜택: **대중교통(지하철, 버스) 이용 시 10% 청구할인, 편의점 10% 할인, 카페 10% 할인, 온라인 쇼핑몰 5% 할인, 영화 및 공연 관람 시 5% 할인, 세탁소 10% 할인, 정육점 10% 할인, 슈퍼마켓 5% 할인, 반찬가게 5% 할인, 야채과일가게 5% 할인, 번개장터(번개페이) 5% 할인, 카드 디자인 선택 가능** 등의 다양한 혜택이 있습니다.
